In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd
import re
from transformers import BertTokenizer, BertTokenizerFast
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertModel
import torch.optim as optim
import numpy as np
from tqdm import tqdm

2025-03-30 21:47:02.159052: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-30 21:47:02.317353: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743351422.375870    4962 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743351422.392920    4962 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-30 21:47:02.544325: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
class BERT_LSTM_GRU(nn.Module):
    def __init__(self, bert_model, hidden_dim, embedding_dim):
        super(BERT_LSTM_GRU, self).__init__()
        self.bert = bert_model
        self.bert.requires_grad_(False)
        self.lstm = nn.LSTM(768, hidden_dim, batch_first=True).to(torch.float32)
        self.gru = nn.GRU(hidden_dim, hidden_dim, batch_first=True).to(torch.float32)
        self.lm_head = nn.Linear(hidden_dim, embedding_dim).to(torch.float32)

    def forward(self, x):
        with torch.no_grad():
            embedding = self.bert(x).last_hidden_state
        x, _ = self.lstm(embedding)
        x, _ = self.gru(x)
        x = self.lm_head(x)
        return x

In [3]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained('bert-base-uncased')
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
model = BERT_LSTM_GRU(bert_model, 512, tokenizer.vocab_size)
checkpoint = torch.load("../models/model_epoch2_step20000.pt", map_location="cpu")

/tmp/ipykernel_4962/507995885.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("../models/model_epoch2_step20000.pt", map_location="cpu")


In [5]:
model.load_state_dict(checkpoint["model_state_dict"])

<All keys matched successfully>

In [6]:
print(checkpoint["loss"])

0.23387010395526886


In [7]:
def generate_text(model, tokenizer, prompt, max_length=50, device=device):
    model.eval()

    tokens = tokenizer.encode(prompt, add_special_tokens=True, return_tensors='pt').to(device)

    generated_tokens = tokens.clone()

    with torch.no_grad():
        for _ in range(max_length):
            logits = model(generated_tokens)

            next_token_logits = logits[:, -1, :]
            next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)

            generated_tokens = torch.cat([generated_tokens, next_token], dim=1)

            if next_token.item() == tokenizer.eos_token_id:
                break

    generated_text = tokenizer.decode(generated_tokens.squeeze(0), skip_special_tokens=True)

    return generated_text

In [8]:
prompt = "Once upon a time, there was a big dog named Max. Max had a red collar that he wore every day. He loved to play and run in the park with his friends."
print(generate_text(model, tokenizer, prompt))

once upon a time, there was a big dog named max. max had a red collar that he wore every day. he loved to play and run in the park with his friends. was was a friendly spot. max loved to play with his warm spot. he always ran after out in the park and had been wag high in all the fun. he was so happy to have found his friends. he was so happy in his


In [10]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/code-
[nltk_data]     dev05/nltk_data...


True

In [12]:
generate_text(model, tokenizer, prompt)

'once upon a time, there was a big dog named max. max had a red collar that he wore every day. he loved to play and run in the park with his friends. was was a friendly spot. max loved to play with his warm spot. he always ran after out in the park and had been wag high in all the fun. he was so happy to have found his friends. he was so happy in his'

In [13]:
from nltk.translate.meteor_score import meteor_score

reference = [prompt.lower().split()]
candidate = generate_text(model, tokenizer, prompt).split()

meteor_score(reference, candidate)

0.7107438016528926

In [14]:
from rouge_score import rouge_scorer

reference = prompt
candidate = generate_text(model, tokenizer, prompt)

scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

scores = scorer.score(reference, candidate)

print(scores)

{'rouge1': Score(precision=0.4230769230769231, recall=1.0, fmeasure=0.5945945945945945), 'rouge2': Score(precision=0.4155844155844156, recall=1.0, fmeasure=0.5871559633027523), 'rougeL': Score(precision=0.4230769230769231, recall=1.0, fmeasure=0.5945945945945945)}
